In [1]:
import time, random, sys
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from opensky_api import OpenSkyApi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from kf.kf import KF
#import smopy
#from scipy.misc import imresize

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


In [3]:
def lat_lon_to_dist(lat_min, lon_min, lat_max, lon_max):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat_min)
    lon1 = radians(lon_min)
    lat2 = radians(lat_max)
    lon2 = radians(lon_max)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c * 1000

In [4]:
def init_kalman(delta_time, q, r):
    dt = delta_time
    A = np.array([
        [1, 0, 0, dt, 0, 0],
        [0, 1, 0, 0, dt, 0],
        [0, 0, 1, 0, 0, dt],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]
    ])
    B = None
    H = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]
    ])
    R = r**2 * np.eye(3)
    Q = q * np.diag([dt, dt, dt, dt, dt, dt])
    kf = KF(A=A, B=B, H=H, R=R, Q=Q)
    return kf

In [5]:
def update_data(s, data, kfs):
    data.append({})
    # Process data.
    for state in s.states:
        # Filter unnamed flights and flights with None altitude.
        if len(state.callsign) < 1: continue
        if state.on_ground is False and state.baro_altitude is None: continue
        if state.on_ground: continue
        
        #lat, lon = mp.to_pixels(state.latitude, state.longitude)
        #lat, lon = state.latitude, state.longitude
        lon = lat_lon_to_dist(lat_min, lon_min, lat_min, state.longitude)
        lat = lat_lon_to_dist(lat_min, lon_min, state.latitude, lon_min)
        
        if state.callsign not in kfs:
            # If the flight is new, create new Kalman filter for it.
            kf = init_kalman(delta_time=1., q=2., r=.5)
            # TODO kf predict?
            kf.predict()
            kfs[state.callsign] = kf
        kf = kfs[state.callsign]
        
        if state.on_ground:
            data[-1][state.callsign] = (lon, lat, 0)
            kf.update(np.array([lon, lat, 0]))
        else:
            kf.update(np.array([lon, lat, state.baro_altitude]))
            data[-1][state.callsign] = (lon, lat, state.baro_altitude)

In [6]:
def predict_kalman(data, kfs):
    # TODO
    data.append({})
    for flight in data[-2]:
        kf = kfs[flight]
        kf.predict()
        data[-1][flight] = (kf.x[0], kf.x[1], kf.x[2])
        print("last:", data[-2][flight])
        print("predicted:", data[-1][flight])

In [7]:
# Prague, (Morina - Sojovice)
lat_min, lon_min = 49.951508, 14.212285
lat_max, lon_max = 50.221933, 14.763462
lat_center, lon_center = 50.101770, 14.263117
lat_min, lon_min = lat_center - 0.2, lon_center - 0.2
lat_max, lon_max = lat_center + 0.2, lon_center + 0.2
airport_name = "Vaclav Havel's Airport"
"""
# Vaclav Havel's Airport
lat_min, lon_min = 50.074975, 14.199871
lat_max, lon_max= 50.130103, 14.324053
lat_center, lon_center = 50.101770, 14.263117
airport_name = "Vaclav Havel's Airport"

# London Heathrow Airport (Sunningdale - Wembley)
lat_min, lon_min = 51.392666, -0.633652
lat_max, lon_max = 51.550291, -0.301164
lat_center, lon_center = 51.467612, -0.453609
airport_name = "London Heathrow Airport"

# Chicago O'Hare International Airport (Wheaton - somewhere in the Michigan lake)
lat_min, lon_min = 41.868163, -88.097664
lat_max, lon_max = 42.163555, -87.601343
lat_center, lon_center = 41.980317, -87.912309
airport_name = "Chicago O'Hare International Airport"
# Chicago O'Hare International Airport (Addison - Park Ridge)
lat_min, lon_min = 41.931889, -87.987572
lat_max, lon_max = 42.011809, -87.834858
lat_center, lon_center = 41.980317, -87.912309
airport_name = "Chicago O'Hare International Airport"
"""
dlon = (lon_max - lon_min)
dlat = (lat_max - lat_min)
ratio = dlon / dlat
api = OpenSkyApi()

In [16]:
################################ Initialize variables ##################################
s, line, prev_time = None, None, None
anns, lst = [], []
d, color_dict, kfs = {}, {}, {}
i, c = 0, 8
kf = False

################################ Initialize figure #####################################
fig = plt.figure(figsize=(c * ratio, c))
ax = fig.add_subplot(111, projection='3d')

#mp = smopy.Map((lat_min, lon_min, lat_max, lon_max))
#img = mp.to_numpy() / 255.0
#x, y = np.ogrid[0:img.shape[0], 0:img.shape[1]]
#ax.plot_surface(x, y, np.zeros(y.shape), rstride=5, cstride=5, facecolors=img)

xlim = lat_lon_to_dist(lat_min, lon_min, lat_min, lon_max)
ylim = lat_lon_to_dist(lat_min, lon_min, lat_max, lon_min)
#ax.set_xlim(0, xlim)
#ax.set_ylim(0, ylim)
ax.set_zlim(0, 2500)

#aa, bb = mp.to_pixels(lat_center, lon_center)
#aa, bb = lat_center, lon_center
bb = lat_lon_to_dist(lat_min, lon_min, lat_min, lon_center)
aa = lat_lon_to_dist(lat_min, lon_min, lat_center, lon_min)
ax.plot([bb], [aa], [0], 'o', c='green', markersize=16, label = airport_name)
plt.legend()

################################ Tracking airplanes #####################################
i = 0
s2 = api.get_states(bbox = (lat_min, lat_max, lon_min, lon_max))
start = s2.time
start_real = int(time.time())
cur = start
num_real_data = 1
MIN_REAL_DATA = 3
while True:
    if cur - start < i:
        # Wait until at least i seconds pass since the start.
        t = ax.text2D(0.05, 0.95, str(i), transform=ax.transAxes, fontsize=14, verticalalignment='top')
        plt.pause(0.01)
        t.remove()
        cur = start + (int(time.time()) - start_real)
        continue
    i += 1
    if i == 1:
        # i == 1, special case of the first iteration.
        update_data(s2, lst, kfs)
    elif num_real_data < MIN_REAL_DATA:
        s2 = api.get_states(bbox = (lat_min, lat_max, lon_min, lon_max))
        if s2 is None:
            continue
        if s2.time == prev_time:
            continue
        num_real_data += 1
        update_data(s2, lst, kfs)
        prev_time = s2.time
    else:
        # num_real_data >= MIN_REAL_DATA
        # Try to get new data from API (updated aproximatelly every 10-th sec).
        s2 = api.get_states(bbox = (lat_min, lat_max, lon_min, lon_max))
        if s2 is None:
            # Predict positions using 3D Kalman filter.
            predict_kalman(lst, kfs)
            #continue
        elif s2.time == prev_time:
            # Predict positions using 3D Kalman filter.
            predict_kalman(lst, kfs)
            #continue
        else:
            # Update our data with obtained real data.
            num_real_data += 1
            update_data(s2, lst, kfs)
            prev_time = s2.time
    
    # Load data for annotations.
    anns_info = []
    for flight in lst[-1]:
        if flight not in color_dict:
            color_dict[flight] = np.random.rand(3,)
        lon, lat, alt = lst[-1][flight]
        anns_info.append((flight, lon, lat, alt))
    # Delete old annotations from the figure.
    while len(anns) > 0:
        anns.pop().remove()

    # Plot figure.
    if len(lst) == 1:
        # Special case of the first iteration.
        for flight in lst[-1]:
            line, = ax.plot([lst[-1][flight][0]], [lst[-1][flight][1]], [lst[-1][flight][2]], 'x', c='black')
        for item in anns_info:
            ann = ax.text(item[1], item[2], item[3], item[0])
            anns.append(ann)
    else:
        # All other iterations.
        for flight in lst[-1]:
            if flight in lst[-2]:
                ax.plot([lst[-2][flight][0], lst[-1][flight][0]], [lst[-2][flight][1], lst[-1][flight][1]], [lst[-2][flight][2], lst[-1][flight][2]], '-x', c=color_dict[flight])
        for item in anns_info:
            ann = ax.text(item[1], item[2], item[3], item[0])
            anns.append(ann)
        plt.draw()
    #plt.legend()

last: (12901.707662864561, 13795.836215467, 1280.16)
predicted: (18776.15804784773, 22800.204490948512, 1762.4353735210566)
last: (11898.704435703045, 22338.287407500266, 571.5)
predicted: (17839.885486817602, 33492.090796511824, 856.857535272818)
last: (19800.933485541296, 26019.994887399724, 647.7)
predicted: (31763.892231142607, 39978.95481147614, 1094.023902850186)
last: (8681.92914239686, 25374.86185466789, 12298.68)
predicted: (19485.16560274068, 34725.25932510111, 18441.30108825274)
last: (18776.15804784773, 22800.204490948512, 1762.4353735210566)
predicted: (25030.707697363963, 30395.209318046618, 2349.522439983767)
last: (17839.885486817602, 33492.090796511824, 856.857535272818)
predicted: (23782.552204611606, 44648.68333365767, 1142.2864277687602)
last: (31763.892231142607, 39978.95481147614, 1094.023902850186)
predicted: (42344.80236809884, 53296.394788650075, 1458.4555826803612)
last: (19485.16560274068, 34725.25932510111, 18441.30108825274)
predicted: (25975.893651621653, 

In [ ]:
from kf.trajectory3D import trajectory3D as t3
dt = 1. # Delta time.
q = 2. # Variance multiplier of x data.
r = .5 # Variance multiplier of y measurements.
traj = t3(2605, delta_time=dt, q=q, r=r)
kf = init_kalman(delta_time=dt, q=q, r=r)
for yt in traj.Y.T:
    kf.predict()
    kf.update(yt)
    kf.log()

log_x = np.array(kf.log_x).T

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.plot(traj.Y[0, :], traj.Y[1, :], traj.Y[2, :], '+')
ax.plot(log_x[0, :], log_x[1, :], log_x[2, :])
ax.plot(traj.X[0, :], traj.X[1, :], traj.X[2, :], c='black')
plt.show()

In [ ]:
if i > 0:
        #lgnd = int((time.time() - start) / 1)
        #t = ax.text2D(0.05, 0.95, str(lgnd), transform=ax.transAxes, fontsize=14, verticalalignment='top')
        plt.pause(0.01)
        #t.remove()

In [ ]:
s = api.get_states(bbox = (lat_min, lat_max, lon_min, lon_max))
if s is not None:
    print(s.time)

In [ ]:
s2 = api.get_states(bbox = (lat_min, lat_max, lon_min, lon_max))
if s2 is not None:
    print(s2.time)